In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_score, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

import math
import joblib 
import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
# TODO
# income level to numerical
# MinMaxScaler()
# Feature importance https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html
# https://www.kaggle.com/residentmario/automated-feature-selection-with-sklearn



In [3]:
# data

# import data
df = pd.read_csv('../../#task4-eda/datasets/OUTPUT_WBI_exposer_cyclones_v5.csv', sep=';')
df.columns = [e.lower().replace(' ','_') for e in df.columns]

# separate X and y
X = df.drop(['total_affected'], axis=1)
y = df['total_affected']

# types of features
num_features = list(df.loc[:, df.dtypes != object].columns)
cat_features = list(df.loc[:, df.dtypes == object].columns)

# fixing types of features
num_features.append('income_level_final')
cat_features.remove('income_level_final')

In [4]:
# custom transformers

class DataFrameSelector(BaseEstimator, TransformerMixin):
    # selects a slice of the dataset based on feature_names
    def __init__(self, feature_names):
        self.feature_names = feature_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[set(X.columns).intersection(self.feature_names)]


class TransformerCat(BaseEstimator, TransformerMixin):
    # transforms categorical features
    
    def __init__(self):
        self.features_to_keep = []
        self.features_to_drop = []

    def fit(self, X, y=None):
        self.features_to_keep = ['basin']
        self.features_to_drop= ['sid', 'name', 'iso', 'sub_basin', 'nature', 'iso_time',
       'coords']
        return self
    
    def _get_income_level(self, income):
        order = {'High': 4, 'High_Middle': 3, 'Low_Middle': 2, 'Low': 1 }
        return order[income]
        
    def transform(self, X, y=None):
        X = X.drop(self.features_to_drop, axis = 1)
        return X
    
class TransformerNum(BaseEstimator, TransformerMixin):
    # transforms numerical features
    
    def __init__(self):
        self.features_to_keep = []
        self.features_to_drop = []
        self.features_to_log = []
    
    def _get_income_level(self, income):
        order = {'High': 4, 'High_Middle': 3, 'Low_Middle': 2, 'Low': 1 }
        return order[income]

    def fit(self, X, y=None):
        # do not drop ['total_hrs', 'wind_calc_mean', 'storm_spd_mean', 'v_land_kn', \
        # 'population_density_(people_per_sq._km_of_land_area)', 'rural_population_(%_of_total_population)', \
        # 'income_level_final', 'pop_max_34', 'pop_max_50', 'pop_max_64'] based on Arnab's Analysis
        
        
        self.features_to_keep = [['96kn_assets', 'cpi', 'pop_max_50', 
                                 'population_density_(people_per_sq._km_of_land_area)', 
                                 'life_expectancy_at_birth,_total_(years)', 
                                 'v_land_kn', 'pres_cal_max', 'wind_cal_max', 'wind_calc_mean', 
                                 'total_hrs', 'storm_spd_mean', 'pop_max_64', 'total_affected', 
                                 'gdp_per_capita_(constant_2010_us$)', 'rural_population_(%_of_total_population)', 
                                 'storm_spd_max', 'pop_max_34', 'pres_calc_mean', 'income_level_final']]
    
        self.features_to_drop = ['usa_sshs', 'year', 'day_hrs', 'night_hrs', '34kn_assets', '64kn_assets', 
                                'total_damage_(000$)', 'total_deaths', 'air_transport,_freight_(million_ton-km)', 
                                'arable_land_(hectares_per_person)', 'cereal_yield_(kg_per_hectare)', 
                                'food_production_index_(2004-2006_=_100)', 'gdp_growth_(annual_%)', 
                                'net_flows_from_un_agencies_us$', 'mobile_cellular_subscriptions_(per_100_people)', 
                                'adjusted_savings:_education_expenditure_(%_of_gni)',
                                'storm_dr_max', 'storm_dr_mean', 
                                'storm_spd_min', 'storm_dr_min', 'pres_cal_min', 'wind_cal_min']
        
        self.features_to_log = ['population_density_(people_per_sq._km_of_land_area)',
                               'pop_max_50', 'pop_max_64']

        return self
    
    def _get_log(self, f):
        try:
            return math.log(f)
        except:
            return 0
        
    def transform(self, X, y=None):
        X['income_level_final'] = X['income_level_final'].apply(self._get_income_level)
        X = X.drop(self.features_to_drop, axis = 1)
        for f in self.features_to_log:
            X[f] = X[f].apply(self._get_log)
        return X

In [5]:
# preprocess pipeline

preprocess_pipeline = FeatureUnion(transformer_list =[
        ("num_pipeline", Pipeline([
            ('selector', DataFrameSelector(num_features)),
            ('num_transformer', TransformerNum()),
            ('imputer', SimpleImputer(strategy="most_frequent")),
            ('scaler', StandardScaler()),
            # ('uniform_distribution', QuantileTransformer(random_state=42)) # decreasing performance
            # ('pca', PCA(0.99)) # decreasing performance
             ])),
        ("cat_pipeline", Pipeline([
            ('selector', DataFrameSelector(cat_features)),
            ('cat_transformer', TransformerCat()),
            ('imputer', SimpleImputer(strategy="most_frequent")),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse = False)), # TODO: not encode income_level_final
            # ("scaler", StandardScaler()) # decreasing performance
            ]))])

In [6]:
class ModelEvaluator:
    # evaluates different models using cross validation on the whole dataset
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.mean_scores = dict()
        
    def evaluate_models(self, models):
        for model_name, model in models.items():
            print(f'***{model_name}***')
            self._evaluate_model(model_name, model)
            print('\n')
            
    def _display_scores(self, scores):
            print("Scores:", scores)
            print("Mean:", scores.mean())
            print("Standard deviation:", scores.std())

    def _evaluate_model(self, model_name, model):
        transformed_X = preprocess_pipeline.fit_transform(self.X)
        scores = cross_val_score(model, transformed_X, self.y, scoring="neg_mean_squared_error", cv=10)
        tree_rmse_scores = np.sqrt(-scores)
        self._display_scores(tree_rmse_scores)
        self.mean_scores[model_name] = tree_rmse_scores.mean()
        
    def save_models(self, models):
        for model_name, model in models.items():
            transformed_X = preprocess_pipeline.fit_transform(self.X)
            model.fit(transformed_X, self.y)
            joblib.dump(model, f'./models/{datetime.datetime.now().date()}_bego_{model_name}_{int(self.mean_scores[model_name])}.pkl')


In [7]:
"""# grid search random_forest

param_grid = [
    {'n_estimators': list(range(10,200,30)), 'max_features': list(range(2, 10, 2)), 'min_samples_leaf':[2,4,6]},
    {'bootstrap': [False], 'n_estimators': list(range(10,200,30)), 'max_features': list(range(2, 10, 2)), 'min_samples_leaf':[2,4,6]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search_forest = GridSearchCV(forest_reg, 
                                  param_grid, 
                                  cv=6, 
                                  scoring='neg_mean_squared_error', 
                                  return_train_score=True, 
                                  iid = True, 
                                  verbose=1) # higher verbose desactivated for Github visualization purposes
transformed_X = preprocess_pipeline.fit_transform(X)
grid_search_forest.fit(transformed_X, y)

print(grid_search_forest.score(transformed_X, y))
print(grid_search_forest.best_params_)""";


In [8]:
### evaluate models 

models = {'forest_1': RandomForestRegressor(n_estimators=3, random_state=42),
          'forest_2':RandomForestRegressor(max_depth=5, n_estimators=10, max_features=9, random_state=42),
          'forest_3':RandomForestRegressor(max_depth=10, n_estimators=20, random_state=42),
          #'forest_grid_search': grid_search_forest.best_estimator_
         }

random_forest_evaluator = ModelEvaluator(X, y)

random_forest_evaluator.evaluate_models(models)

random_forest_evaluator.save_models(models)

***forest_1***
Scores: [1370135.30357728 1467502.85375128 1545967.82330743 2033419.86963731
 1681181.03382911 3364613.62252863 1424137.54499329 1499352.50608047
 2344327.8899283  2884502.59315723]
Mean: 1961514.1040790346
Standard deviation: 657245.8516561881


***forest_2***
Scores: [1041653.16647244 1186679.68707165 1395298.67407935 2029581.5359116
 1383421.08741117 3632794.22043755 1313248.38016145 1296766.38022253
 2228743.93373959 2115604.62084664]
Mean: 1762379.1686353975
Standard deviation: 738908.7087909199


***forest_3***
Scores: [1400176.31317098 1214628.35309767 1411254.04912044 2008843.04925284
 1358379.94912634 3368863.11161206 1344335.77271017 1249713.77227627
 1996216.52470632 2636167.89938798]
Mean: 1798857.879446108
Standard deviation: 678360.3001354883


